In [1]:
# Import the necessary libraries
import wandb
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv("WANDB_API_KEY")

# Login to Weights & Biases using the API key
try:
    wandb.login(key=api_key)
    print("Logged in successfully.")
except Exception as e:
    print(f"Error during login: {e}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ravikumarchavva (ravikumarchavva-org). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\chavv\_netrc


Logged in successfully.


In [2]:
# Updated Configuration
CONFIGURATION = {
    'BATCH_SIZE': 32,
    'IM_SIZE': 224,
    'N_EPOCHS': 10,
    'LEARNING_RATE': 1e-5,
    'NUM_CLASSES': 3,
}

run = wandb.init(
    # Set the project where this run will be logged
    project="human-emotion-estimation-pytorch",

    # Set the experiment name
    name="human-emotion-estimation-1",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": CONFIGURATION['LEARNING_RATE'],
        "epochs": CONFIGURATION['N_EPOCHS'],
        "batch_size": CONFIGURATION['BATCH_SIZE'],
        "image_size": CONFIGURATION['IM_SIZE'],
        "num_classes": CONFIGURATION['NUM_CLASSES'],
    },
)

In [3]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
TRAIN_DIR = '../../EmotionsDataset/train/'
TEST_DIR = '../../EmotionsDataset/test/'
CLASS_NAMES = ['angry','happy','sad']

class EmotionsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.dataset = ImageFolder(root=root_dir, transform=transform)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return image, label

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
train_dataset = EmotionsDataset(TRAIN_DIR, transform=transform)
test_dataset = EmotionsDataset(TEST_DIR, transform=transform)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
import torch
import torch.nn as nn
import timm

class EmotionsModel(nn.Module):
    def __init__(self, num_classes=3):
        super(EmotionsModel, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.model.children())[:-1])

        # The final feature map size for EfficientNet-B0 is 1280
        self.classifier = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.features(x)

        # Flatten the output from the feature extractor
        x = x.view(x.size(0), -1)

        x = self.classifier(x)
        return x

# Instantiate the model
model = EmotionsModel()

# Summarize the model architecture
from torchinfo import summary
summary(model, input_size=(1, 3, 224, 224))  # Input shape is (batch_size, channels, height, width)


Layer (type:depth-idx)                             Output Shape              Param #
EmotionsModel                                      [1, 3]                    1,281,000
├─Sequential: 1-1                                  [1, 1280]                 --
│    └─Conv2d: 2-1                                 [1, 32, 112, 112]         864
│    └─BatchNormAct2d: 2-2                         [1, 32, 112, 112]         64
│    │    └─Identity: 3-1                          [1, 32, 112, 112]         --
│    │    └─SiLU: 3-2                              [1, 32, 112, 112]         --
│    └─Sequential: 2-3                             [1, 320, 7, 7]            --
│    │    └─Sequential: 3-3                        [1, 16, 112, 112]         1,448
│    │    └─Sequential: 3-4                        [1, 24, 56, 56]           16,714
│    │    └─Sequential: 3-5                        [1, 40, 28, 28]           46,640
│    │    └─Sequential: 3-6                        [1, 80, 14, 14]           242,930
│    │    └

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import wandb
from sklearn.metrics import confusion_matrix, accuracy_score


N_EPOCHS = 5
LEARNING_RATE = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

model.to(device)
criterion.to(device)

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0

    all_labels = []
    all_preds = []
    logged_images = []

    for i, (images, labels) in enumerate(tqdm(dataloader, desc="Training", leave=False)):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct_preds += (preds == labels).sum().item()
        total_samples += labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

        # Log sample images (e.g., first 5 batches)
        if i < 5:
            for img, label, pred in zip(images[:5], labels[:5], preds[:5]):
                img = img.permute(1, 2, 0).cpu().numpy()  # Convert image from Tensor to numpy (HWC format)
                logged_images.append(wandb.Image(img, caption=f"True: {label.item()}, Pred: {pred.item()}"))

    avg_loss = running_loss / len(dataloader)
    accuracy = correct_preds / total_samples
    conf_matrix = confusion_matrix(all_labels, all_preds)

    return avg_loss, accuracy, conf_matrix, logged_images

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0

    all_labels = []
    all_preds = []
    logged_images = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(dataloader, desc="Evaluating", leave=False)):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct_preds += (preds == labels).sum().item()
            total_samples += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

            # Log sample images (e.g., first 5 batches)
            if i < 5:
                for img, label, pred in zip(images[:5], labels[:5], preds[:5]):
                    img = img.permute(1, 2, 0).cpu().numpy()  # Convert image from Tensor to numpy (HWC format)
                    logged_images.append(wandb.Image(img, caption=f"True: {label.item()}, Pred: {pred.item()}"))

    avg_loss = running_loss / len(dataloader)
    accuracy = correct_preds / total_samples
    conf_matrix = confusion_matrix(all_labels, all_preds)

    return avg_loss, accuracy, conf_matrix, logged_images

# Example usage
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch + 1}/{N_EPOCHS}")
    train_loss, train_accuracy, train_conf_matrix, train_images = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_accuracy, val_conf_matrix, val_images = evaluate(model, test_dataloader, criterion, device)

    # Log metrics and images to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "train_conf_matrix": wandb.plot.confusion_matrix(probs=None, y_true=[int(x) for row in train_conf_matrix for x in row], preds=[int(x) for row in train_conf_matrix for x in row]),
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "val_conf_matrix": wandb.plot.confusion_matrix(probs=None, y_true=[int(x) for row in val_conf_matrix for x in row], preds=[int(x) for row in val_conf_matrix for x in row]),
        "train_images": train_images,  # Log images from training
        "val_images": val_images  # Log images from validation
    })

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/5


Train Loss: 0.5691, Train Accuracy: 0.7588
Validation Loss: 0.4214, Validation Accuracy: 0.8570
Epoch 2/5


Train Loss: 0.3614, Train Accuracy: 0.8587
Validation Loss: 0.3965, Validation Accuracy: 0.8518
Epoch 3/5


Train Loss: 0.2740, Train Accuracy: 0.8945
Validation Loss: 0.4165, Validation Accuracy: 0.8434
Epoch 4/5


Train Loss: 0.2369, Train Accuracy: 0.9084
Validation Loss: 0.3974, Validation Accuracy: 0.8575
Epoch 5/5


Train Loss: 0.1825, Train Accuracy: 0.9290
Validation Loss: 0.3817, Validation Accuracy: 0.8803
